In [71]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Runs

In [72]:
run = "VAL_naive_doctaet-few_shot_template_initial-mixtral8x7b-20240613-141202"

run.rsplit("-", 4)[1], run.rsplit("-", 4)[2], run

('few_shot_template_initial',
 'mixtral8x7b',
 'VAL_naive_doctaet-few_shot_template_initial-mixtral8x7b-20240613-141202')

In [73]:
run.rsplit("-", 4)

['VAL_naive_doctaet',
 'few_shot_template_initial',
 'mixtral8x7b',
 '20240613',
 '141202']

In [77]:
from src.content_extraction import format
import pandas as pd

runs = [
    # "VAL_naive_doctaet-simple_zs-mistralaiMistral7BInstructv03-20240611-095123",
    # "VAL_naive_doctaet-simple_fs-mistralaiMistral7BInstructv03-20240611-102154",
    # "VAL_naive_doctaet-simple_zs-googlegemma7bit-20240611-130659",
    # "VAL_naive_doctaet-simple_fs-googlegemma7bit-20240611-133609",
    # "VAL_llama3_70b_fewshot_optimized02-20240531-105142",
    # "VAL_llama3_8b_few_shot_template_initial-20240604-125136",
    # "VAL_naive_doctaet-simple_zs-llama38b-20240611-212642",
    # "VAL_naive_doctaet-simple_fs-llama38b-20240611-213146",
    # "VAL_naive_doctaet-simple_zs-gemma7b-20240612-073229",
    # "VAL_naive_doctaet-simple_fs-gemma7b-20240612-073907",
    # "VAL_naive_doctaet-simple_zs-mistral7b-20240612-074427",
    # "VAL_naive_doctaet-simple_fs-mistral7b-20240612-075256",
    # "VAL_naive_doctaet-simple_zs-llama370b-20240612-083003",
    # "VAL_naive_doctaet-simple_fs-llama370b-20240612-085839",
    # "VAL_naive_doctaet-simple_fs_v2-llama370b-20240612-121227",
    "VAL_naive_doctaet-few_shot_template_initial-llama370b-20240612-132023",
    # "VAL_naive_doctaet-simple_fs_v2-llama370b-20240612-141913",
    # "VAL_naive_doctaet-simple_fs_v2-llama38b-20240612-151333",
    # "VAL_naive_doctaet-simple_fs_v3-llama38b-20240612-152251",
    # "VAL_naive_doctaet-simple_fs_v2-mistral7b-20240612-153111",
    # "VAL_naive_doctaet-simple_fs_v3-mistral7b-20240612-154022",
    # "VAL_naive_doctaet-simple_fs_v2-gemma7b-20240612-154846",
    # "VAL_naive_doctaet-simple_fs_v3-gemma7b-20240612-155251",
    # "VAL_naive_doctaet-simple_fs_v3-llama370b-20240612-155840",
    "VAL_naive_doctaet-simple_fs_v4a-llama38b-20240612-205302",
    "VAL_naive_doctaet-simple_fs_v4b-llama38b-20240612-210144",
    "VAL_naive_doctaet-simple_fs_v4a-mistral7b-20240613-120035",
    "VAL_naive_doctaet-simple_fs_v4b-mistral7b-20240613-120845",
    # "VAL_naive_doctaet-simple_fs_v4a-gemma7b-20240613-121703",
    # "VAL_naive_doctaet-simple_fs_v4b-gemma7b-20240613-122128",
    # "VAL_naive_doctaet-simple_fs_v4a-gemma2b-20240613-122600",
    # "VAL_naive_doctaet-simple_fs_v4b-gemma2b-20240613-123031",
    "VAL_naive_doctaet-few_shot_template_initial-mixtral8x7b-20240613-141202",
    "VAL_naive_doctaet-simple_fs_v4a-mixtral8x7b-20240613-142743",
    "VAL_naive_doctaet-simple_fs_v4b-mixtral8x7b-20240613-144155",
    "VAL_naive_doctaet-few_shot_template_initial-mixtral8x22b-20240613-145709",
    "VAL_naive_doctaet-simple_fs_v4a-mixtral8x22b-20240613-154052",
    "VAL_naive_doctaet-simple_fs_v4b-mixtral8x22b-20240613-162724",
]



def load_run(run):
    template, model = run.rsplit("-", 4)[1], run.rsplit("-", 4)[2]
    path = f"results/{run}/df.feather"

    df = pd.read_feather(path)
    df["model"] = model
    df["prompt_template"] = template
    try:
        df = df.rename({"annontation": "annotation"}, axis=1) # typo in early versions
    except:
        pass
    return df

def post_proc(df):
    df["annotation"] = df["annotation"].apply(format)
    df["annotation"] = df["annotation"].astype(pd.StringDtype())
    df["ground_truth"] = df["ground_truth"].astype(pd.StringDtype())
    return df

dfs = []

for run in runs:
    df = load_run(run)
    dfs.append(df)


# Some older runs might not be in the right format -> format function is idempotent
for df in dfs:
    df = post_proc(df)



In [75]:
df_all = pd.concat(dfs)
df_all

,run,f,annotation,prompt,raw,ground_truth,inference_s,model,prompt_template
0,VAL_naive_doctaet-simple_fs_v4a-llama38b-20240...,1503.05062,unanswerable,"If the text reports benchmark results, extract...",No entries were found in the provided text. Th...,unanswerable,6.582750,llama38b,simple_fs_v4a
1,VAL_naive_doctaet-simple_fs_v4a-llama38b-20240...,1109.0784,unanswerable,"If the text reports benchmark results, extract...",Since the provided text does not report benchm...,unanswerable,0.825765,llama38b,simple_fs_v4a
2,VAL_naive_doctaet-simple_fs_v4a-llama38b-20240...,2210.16422v1,[ {'LEADERBOARD':{'Task': 'Long document extra...,"If the text reports benchmark results, extract...","Based on the provided text, I extracted the fo...",[{'LEADERBOARD': {'Task': 'Text Summarization'...,9.037052,llama38b,simple_fs_v4a
3,VAL_naive_doctaet-simple_fs_v4a-llama38b-20240...,2310.07488v2,"[ {'LEADERBOARD':{ 'Task': 'KMath', 'Dataset':...","If the text reports benchmark results, extract...","Based on the provided text, I extracted the re...",[{'LEADERBOARD': {'Task': 'Arithmetic Reasonin...,5.994259,llama38b,simple_fs_v4a
4,VAL_naive_doctaet-simple_fs_v4a-llama38b-20240...,2210.15425v1,[{'LEADERBOARD':{'Task': 'Facial Expression Re...,"If the text reports benchmark results, extract...",Here are the extracted leaderboard entries:\n\...,"[{'LEADERBOARD': {'Task': 'Keyword Spotting', ...",3.954267,llama38b,simple_fs_v4a
...,...,...,...,...,...,...,...,...,...
95,VAL_naive_doctaet-simple_fs_v4b-mixtral8x22b-2...,2006.09264v3,[{'LEADERBOARD':{'Task': 'One-shot Neural Arch...,"If the text reports benchmark results, extract...","[\n {\n ""Task"": ""One-shot Neural Arch...",[{'LEADERBOARD': {'Task': 'Neural Architecture...,8.003138,mixtral8x22b,simple_fs_v4b
96,VAL_naive_doctaet-simple_fs_v4b-mixtral8x22b-2...,1304.2639,unanswerable,"If the text reports benchmark results, extract...",[]\n\nThe provided text does not report any b...,unanswerable,2.839139,mixtral8x22b,simple_fs_v4b
97,VAL_naive_doctaet-simple_fs_v4b-mixtral8x22b-2...,1803.10683v3,[{'LEADERBOARD':{'Task': 'Human Instance Segme...,"If the text reports benchmark results, extract...","[\n {\n ""Task"": ""Human Instance Segme...","[{'LEADERBOARD': {'Task': 'Pose Estimation', '...",49.058732,mixtral8x22b,simple_fs_v4b
98,VAL_naive_doctaet-simple_fs_v4b-mixtral8x22b-2...,2006.06936v2,[{'LEADERBOARD':{'Task': 'Neural Architecture ...,"If the text reports benchmark results, extract...","[\n {\n ""Task"": ""Neural Architecture ...",[{'LEADERBOARD': {'Task': 'Neural Architecture...,28.323156,mixtral8x22b,simple_fs_v4b


# Evaluation

In [76]:
# evaluate
from scoring_program.evaluation import Metrics

results = []
for df in dfs:
    labels = list(df["ground_truth"])
    preds = list(df["annotation"])
    assert len(labels) == len(preds)
    result = Metrics.evaluate_property_wise_json_based(
        label_list=labels, prediction_list=preds
    )

    result.update(
        Metrics.evaluate_rouge(label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"]))
    )
    result["run"] = df["run"].iloc[0]
    result["score"] = result["partial_f1s_overall"]
    result["num_samples"] = len(df)
    try:
        result["mean_t"] = df["inference_s"].mean()
    except:
        pass
    results.append(pd.DataFrame([result]))

df_res = pd.concat(results)
df_res

,general_accuracy,exact_recalls_task,exact_recalls_dataset,exact_recalls_metric,exact_recalls_Score,exact_recalls_overall,partial_recalls_task,partial_recalls_dataset,partial_recalls_metric,partial_recalls_Score,...,partial_f1s_Score,partial_f1s_overall,rouge1,rouge2,rougeL,rougeLsum,run,score,num_samples,mean_t
0,71.0,2.46,19.70,17.24,6.40,11.45,7.88,25.12,20.69,7.28,...,6.76,13.58,38.31,9.60,37.17,37.15,VAL_naive_doctaet-simple_fs_v4a-llama38b-20240...,13.58,100,5.217605
0,79.0,5.91,15.27,14.29,3.94,9.85,8.37,19.70,18.72,5.34,...,4.87,11.72,45.55,9.38,44.00,44.08,VAL_naive_doctaet-simple_fs_v4b-llama38b-20240...,11.72,100,5.274975
0,51.0,11.82,20.20,17.24,9.85,14.78,19.21,22.66,21.67,10.19,...,8.81,14.33,19.54,12.06,18.74,18.48,VAL_naive_doctaet-simple_fs_v4a-mistral7b-2024...,14.33,100,4.891713
0,51.0,8.37,16.26,11.33,10.34,11.58,11.33,18.72,14.78,10.19,...,9.05,10.98,19.38,11.31,18.34,18.14,VAL_naive_doctaet-simple_fs_v4b-mistral7b-2024...,10.98,100,4.968778
0,52.0,17.73,29.56,21.67,14.78,20.94,21.18,30.54,24.63,15.05,...,10.26,15.52,20.70,13.28,19.57,19.15,VAL_naive_doctaet-few_shot_template_initial-mi...,15.52,100,9.401801
0,81.0,13.79,25.12,16.75,10.84,16.63,18.72,26.11,19.70,11.17,...,9.47,16.33,51.63,12.38,50.38,50.90,VAL_naive_doctaet-simple_fs_v4a-mixtral8x7b-20...,16.33,100,8.504601
0,82.0,15.76,22.66,20.20,14.29,18.23,22.17,26.60,26.60,15.05,...,9.31,14.07,52.43,12.44,51.04,51.19,VAL_naive_doctaet-simple_fs_v4b-mixtral8x7b-20...,14.07,100,9.130037
0,53.0,18.72,25.12,22.17,17.73,20.94,26.11,28.57,27.59,18.93,...,13.81,17.41,21.69,12.79,20.71,20.57,VAL_naive_doctaet-few_shot_template_initial-mi...,17.41,100,26.215657
0,78.0,18.23,20.69,18.23,20.69,19.46,29.06,27.59,20.20,21.36,...,16.12,19.04,47.63,12.88,46.47,46.72,VAL_naive_doctaet-simple_fs_v4a-mixtral8x22b-2...,19.04,100,27.911258
0,78.0,16.26,19.21,18.72,11.82,16.50,20.69,24.63,24.14,12.62,...,10.00,16.00,47.52,12.40,46.51,46.53,VAL_naive_doctaet-simple_fs_v4b-mixtral8x22b-2...,16.00,100,74.403017


# Runs on Test

In [79]:

runs = [
    "TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-20240613-191825"
]

df = load_run(runs[-1])
df

,run,f,annotation,prompt,raw,ground_truth,inference_s,model,prompt_template
0,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,0806.2287,[],"If the text reports benchmark results, extract...",[ ]\n\nThe provided text does not contain any...,None,43.949223,mixtral8x22b,simple_fs_v4a
1,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1412.3670,[{'LEADERBOARD': {'Task': 'Facial Expression R...,"If the text reports benchmark results, extract...","Based on the provided text, here are the extr...",None,7.739378,mixtral8x22b,simple_fs_v4a
2,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1001.3451,[],"If the text reports benchmark results, extract...",[ ]\n\nThe provided text does not contain any...,None,4.223240,mixtral8x22b,simple_fs_v4a
3,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1907.05888v1,[{'LEADERBOARD': {'Task': 'Congestive Heart Fa...,"If the text reports benchmark results, extract...","[\n {""Task"": ""Congestive Heart Failure Pre...",None,12.583682,mixtral8x22b,simple_fs_v4a
4,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1911.12667v3,[{'LEADERBOARD': {'Task': 'Action Recognition'...,"If the text reports benchmark results, extract...","[\n {""Task"": ""Action Recognition"", ""Datase...",None,31.150235,mixtral8x22b,simple_fs_v4a
...,...,...,...,...,...,...,...,...,...
784,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,2010.14022v2,[{'LEADERBOARD': {'Task': 'Cover Song Identifi...,"If the text reports benchmark results, extract...","[\n {""Task"": ""Cover Song Identification (C...",None,45.126313,mixtral8x22b,simple_fs_v4a
785,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1210.0408,[],"If the text reports benchmark results, extract...",[]\n\nThe provided text does not contain any ...,None,3.252135,mixtral8x22b,simple_fs_v4a
786,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1604.03799,[],"If the text reports benchmark results, extract...",[ ]\n\nThe provided text does not contain any...,None,3.805816,mixtral8x22b,simple_fs_v4a
787,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,0911.1502,"[{'LEADERBOARD': {'Task': ""Maximizing Service ...","If the text reports benchmark results, extract...","[\n {\n ""Task"": ""Maximizing Service P...",None,26.862081,mixtral8x22b,simple_fs_v4a


# Rerun post processing



In [86]:
from src.dataset import LogResult


orig_run_id = "TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-20240613-191825"
from datetime import datetime
now_str = datetime.now().strftime('%Y%m%d-%H%M%S')

df = load_run(orig_run_id)


run_id = orig_run_id.rsplit("-", 2)[0] + "-" + now_str



# logger.log(f, str(processed), str(prompt), str(model_output), str(ground_truth), elapsed_time)

In [109]:
from src.content_extraction import format

logger = LogResult(run_id, save_interval=100, do_write=False, additional_col_names=["prompt", "raw", "ground_truth", "inference_s", "original_postproc"])
for i, row in df.iterrows():
    processed = format(row["raw"])
    logger.log(row["f"], str(processed), row["prompt"], row["raw"], row["ground_truth"], row["inference_s"], row["annotation"])
    pass

df = logger.save()
df

,run,f,annotation,prompt,raw,ground_truth,inference_s,original_postproc
0,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,0806.2287,[],"If the text reports benchmark results, extract...",[ ]\n\nThe provided text does not contain any...,None,43.949223,[]
1,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1412.3670,[{'LEADERBOARD': {'Task': 'Facial Expression R...,"If the text reports benchmark results, extract...","Based on the provided text, here are the extr...",None,7.739378,[{'LEADERBOARD': {'Task': 'Facial Expression R...
2,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1001.3451,[],"If the text reports benchmark results, extract...",[ ]\n\nThe provided text does not contain any...,None,4.223240,[]
3,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1907.05888v1,[{'LEADERBOARD': {'Task': 'Congestive Heart Fa...,"If the text reports benchmark results, extract...","[\n {""Task"": ""Congestive Heart Failure Pre...",None,12.583682,[{'LEADERBOARD': {'Task': 'Congestive Heart Fa...
4,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1911.12667v3,[{'LEADERBOARD': {'Task': 'Action Recognition'...,"If the text reports benchmark results, extract...","[\n {""Task"": ""Action Recognition"", ""Datase...",None,31.150235,[{'LEADERBOARD': {'Task': 'Action Recognition'...
...,...,...,...,...,...,...,...,...
784,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,2010.14022v2,[{'LEADERBOARD': {'Task': 'Cover Song Identifi...,"If the text reports benchmark results, extract...","[\n {""Task"": ""Cover Song Identification (C...",None,45.126313,[{'LEADERBOARD': {'Task': 'Cover Song Identifi...
785,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1210.0408,[],"If the text reports benchmark results, extract...",[]\n\nThe provided text does not contain any ...,None,3.252135,[]
786,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,1604.03799,[],"If the text reports benchmark results, extract...",[ ]\n\nThe provided text does not contain any...,None,3.805816,[]
787,TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...,0911.1502,"[{'LEADERBOARD': {'Task': ""Maximizing Service ...","If the text reports benchmark results, extract...","[\n {\n ""Task"": ""Maximizing Service P...",None,26.862081,"[{'LEADERBOARD': {'Task': ""Maximizing Service ..."


In [108]:
row

run                TEST_naive_doctaet-simple_fs_v4a-mixtral8x22b-...
f                                                          1406.1244
annotation                                                        []
prompt             If the text reports benchmark results, extract...
raw                 [ ]\n\nThe provided text does not contain any...
ground_truth                                                    None
inference_s                                                  7.82347
model                                                   mixtral8x22b
prompt_template                                        simple_fs_v4a
Name: 788, dtype: object